In [3]:
import cv2
import numpy as np

# Load the image
image = cv2.imread('IMG_6229.JPG')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Apply edge detection
edges = cv2.Canny(gray, 50, 150)

# Find contours
contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
contours = sorted(contours, key=cv2.contourArea, reverse=True)  # Sort by area

# Assume the largest contour is the sheet
sheet_contour = contours[0]

# Approximate the sheet's contour to a quadrilateral
epsilon = 0.02 * cv2.arcLength(sheet_contour, True)
approx = cv2.approxPolyDP(sheet_contour, epsilon, True)

# Ensure we have four points (a quadrilateral)
if len(approx) == 4:
    # Sort the points in the required order
    points = approx.reshape(4, 2)
    rect = np.zeros((4, 2), dtype='float32')

    s = points.sum(axis=1)
    rect[0] = points[np.argmin(s)]  # Top-left
    rect[2] = points[np.argmax(s)]  # Bottom-right

    diff = np.diff(points, axis=1)
    rect[1] = points[np.argmin(diff)]  # Top-right
    rect[3] = points[np.argmax(diff)]  # Bottom-left

    # Define the standard rectangle for the aligned sheet
    (width, height) = (500, 700)  # Adjust these values based on the sheet's size
    dst = np.array([
        [0, 0],
        [width - 1, 0],
        [width - 1, height - 1],
        [0, height - 1]
    ], dtype='float32')

    # Compute the perspective transform matrix
    M = cv2.getPerspectiveTransform(rect, dst)

    # Apply perspective warp
    aligned_sheet = cv2.warpPerspective(image, M, (width, height))

    # Display the aligned sheet
    cv2.imshow('Aligned Sheet', aligned_sheet)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
else:
    print("Sheet alignment failed: Unable to detect 4 corners.")
